# PhantomWiki

PhantomWiki is an on-demand random generator of fictional worlds. Similarly to the wiki hosting services popular in film, video games, and literature [1], we represent these fictional worlds through Wikipedia-like biographical entries about their characters. We then test the model’s retrieval skills and its understanding of the fictional world through an accompanying set of automatically generated question-answer pairs.

<p align="center">
    <img src="assets/PhantomWiki-pipeline.png" alt="PhantomWiki Pipeline"/>
</p>

### 1. Generate a PhantomWiki Instance

The first step of the PhantomWiki pipeline is to generate a random universe of characters as well as the document corpus describing it. 

Each character in a PhantomWiki universe is described through its social relationships and personal facts. For the social relationships, we first generate family trees, following the family tree generator of Hohenecker & Lukasiewicz (2020)[2].

This family tree generation algorithm allows us to specify the parameters (for more please refer to our source code) for family tree generation: 
- `num-family-trees`: the number of family trees in a PhantomWiki universe 
- `max-family-tree-size`: the maximum number of people in one family tree 

These two parameters together determine the size of the universe, i.e., the number of characters: `universe-size = num-family-trees * max-family-tree-size`.

In [ ]:
num_family_trees = 1
max_family_tree_size = 25
# universe size is 1 * 25 = 25

We next specify the complexity of questions generated about this universe with `question-depth`. Questions are generated from question templates, obtained by recursively unrolling the Context-Free Grammar. Here, `question-depth` determines the maximum number of times the CFG will be unrolled. The difficulty of generated questions is proportional to this parameter.

In [ ]:
question_depth = 10

Specify the folder and subfolder (for different splits) you want to store your PhantomWiki instance (including facts, QA pairs.)

In [ ]:
import os

output_dir = "out"
split = "split0"
split_dir = os.path.join(output_dir, split)

Now we can run the following command to generate a PhantomWiki universe. 

In [ ]:
!pip install phantom-wiki

In [ ]:
!phantom-wiki-generate --output-dir $split_dir --question-depth $question_depth --num-family-trees $num_family_trees --max-family-tree-size $max_family_tree_size --article-format json --question-format json --debug

### 2. Visualization of the PhantomWiki universe

Now we have generated a universe with PhantomWiki stored in the `$output_dir` folder. 

#### 2.1 Visualization of family trees

We can first take a look at the family trees we generated. 

In [ ]:
# By default we are showing the first family tree generated, although more may be generated at the last step.
family_tree_file = f"{split_dir}/family_tree_1.png"
from IPython.display import Image

Image(filename=family_tree_file)

Every person in PhantomWiki has a first name and a last name. Colors indicate the gender of people in the PhantomWiki universe. Arrows indicate parental relationship. 

#### 2.2 Generated Articles

The facts generated besides the family relationships include friend relationships, hobbies, occupations for the people in the universe. These facts are stored in `facts.pl` and used when converted into articles. 

Those facts are converted into articles for everyone using pre-defined templates. 
Articles generated are saved in `articles.json`. Each person has an entry in the JSON file with the article's title, content, and their associated facts. (For family relationships only the parents and siblings information are mentioned in the articles.)

Here we can take a look at an example of generated articles:


In [ ]:
# input the name of the person you want to read the article about
import json

name = "Aida Wang"
article_file = f"{output_dir}/articles.json"
with open(article_file, "r") as f:
    article_file = json.load(f)
    for entry in article_file:
        if entry["title"] == name:
            article = entry["article"]
            print(article)

#### 2.3 Generated QA pairs

Each PhantomWiki instance also contains Question-Answer pairs that are consistent with the generated facts. 

According to our CFG's construction, using `--question-depth 10` gives us `8` types of question templates. The number of questions generated from each type of template can be specified via `--num-questions-per-type` (default is `10`). These questions are stored in `questions.json` arranged by template type. 

Let's now look at some of the questions: 

In [ ]:
# specify the type of question you want to look at
type = 0
question_file = f"{output_dir}/questions.json"
import json

with open(question_file, "r") as f:
    questions = json.load(f)

we can look at the result of a sampled question and its answer along with the original question template:

In [ ]:
question = questions[0]
print("Question: ", question["question"])
print("Answer: ", question["answer"])
print("Prolog:", question["prolog"])

The `prolog` key shows the prolog query needed to get the answer of a certain question. For users who have more interest in Prolog, please refer to [3]. 

### 3. Evaluation on a PhantomWiki instance

#### 3.1 Run evaluation with specific model and method

In [ ]:
# Install the evaluation dependencies
!pip install phantom-wiki[eval]

Loading a dataset from HuggingFace or from a local folder: 

In [ ]:
from phantom_eval.utils import load_data

# The dataset we just generated can be loaded by
dataset = load_data(dataset=output_dir, split=split, from_local=True)

# For convenience of development, we provide pre-generated datasets on Huggingface
HF_version = "kilian-group/phantom-wiki-v1"
HF_split = "depth_20_size_50_seed_1"
dataset = load_data(dataset=HF_version, split=HF_split, from_local=False)

Now we can finally run evaluation on the generated PhantomWiki dataset. As an example, we test the `zeroshot` method using a gpt model.

In [ ]:
method = "zeroshot"
model = "gpt-4o-mini-2024-07-18"
preds_dir = "out-evals"

In [ ]:
# to run evaluation on the generated PhantomWiki instance
# !python -m phantom_eval --method $method -od $preds_dir -m $model --from_local --dataset $output_dir --split_list $split

# to run evaluation on a HuggingFace Dataset
!python -m phantom_eval --method $method -od $preds_dir -m $model --dataset $HF_version --split_list $HF_split

#### 3.2 Visualize the reasoning evaluation

Now we can summarize the evaluation results. Moreover, we can also visualize LLM performance as a function of question difficulty. Since the dataset contains questions of varying difficulty (i.e. requiring different number of reasoning steps to answer), we can visualize F1 score v Reasoning Steps.

In [ ]:
!python eval/format_leaderboard.py -od $preds_dir --dataset $HF_version --method_list $method --model_list $model

!python eval/plot_reasoning.py -od $preds_dir --dataset $HF_version --model_list $model

## References
- [1] For example, see stardewvalley.fandom.com or harrypotter.fandom.com.
- [2] Hohenecker, Patrick, and Thomas Lukasiewicz. "Ontology reasoning with deep neural networks." Journal of Artificial Intelligence Research 68 (2020): 503-540.
- [3] Sterling, Leon, and Ehud Y. Shapiro. The art of Prolog: advanced programming techniques. MIT press, 1994. 
